# Simulation eines Matched-Filter Empfängers

Systemmodell: BPSK Quelle -> RRC Pulsformung -> AWGN Kanal -> Matched Filter -> Abtastung


## Generierung der BPSK Symbole und Upsampling

In [14]:
%matplotlib qt

import random
import numpy as np
import matplotlib.pyplot as plt

import rrc

N = 1024*16  # number of symbols


M = 64       # samples per symbol (oversampling factor)
K = 32       # length of impulse response in symbols

rolloff = 1e-3 # RRC Rolloff factor

SNR = 15   # SNR in dB


## Generierung des RRC Pulses

In [39]:
from matplotlib import rc
import matplotlib
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 22})


N_rrc = M*K+1
group_delay = int((N_rrc-1)/2)
g = rrc.get_rrc_ir(N_rrc, M, 1, rolloff)
#g /= np.sqrt(np.sum(g**2))

plt.subplots(2)
plt.subplot(121)
Kplot = min(K//2, 4)
plt.plot(np.arange(-Kplot, Kplot, 1/M), g[len(g)//2-Kplot*M: len(g)//2+Kplot*M], label='Sincpuls', linewidth=2);
rect = np.zeros(M*K)
rect[M*K/2-M/2: M*K/2+M/2] = 1
plt.plot(np.linspace(-K/2, K/2, M*K), rect, label='Rechteckpuls', linewidth=2)
plt.xlabel('$t/T$')
plt.legend()
plt.xlim(-Kplot, Kplot)
plt.title("$g(t)$", fontsize=26); plt.grid()

plt.subplot(122)
f = np.linspace(-M/2, M/2, len(g))
G = np.abs(np.fft.fftshift(np.fft.fft(g))**2/len(g))
plt.semilogy(f, G, label='Sincpuls', linewidth=2)
f = np.linspace(-M/2, M/2, len(rect))
Rect = np.abs(np.fft.fftshift(np.fft.fft(rect))**2/len(rect))
plt.semilogy(f, Rect, label='Rechteckpuls', linewidth=2)
plt.title('$|G(f)|^2$', fontsize=26)
plt.xlabel('$f/B$')
plt.xlim(-4, 4)
plt.legend()
plt.grid()
plt.ylim(1e-3, 10)
plt.show()


/home/wunsch/miniconda3/envs/jupylab/lib/python3.5/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [5]:
N_rrc = M*K+1
group_delay = int((N_rrc-1)/2)
g = rrc.get_rrc_ir(N_rrc, M, 1, rolloff)
g /= np.sqrt(np.sum(g**2))

Kplot = min(K//2, 4)
plt.plot(np.arange(-Kplot, Kplot, 1/M), g[len(g)//2-Kplot*M: len(g)//2+Kplot*M]); 
plt.xlabel('t/T')
plt.title("h(t) RRC pulse"); plt.grid(); plt.plot()

[]

## Generierung des RC Pulses durch Faltung

In [ ]:
gh = np.convolve(g, g, mode='full')

plt.plot(np.arange(-Kplot, Kplot, 1/M), gh[len(gh)//2-Kplot*M: len(gh)//2+Kplot*M]); 
plt.title("g(t)*h(t) RC pulse"); plt.xlabel('T'); plt.grid(); plt.plot()

## Faltung der Sendesymbole mit dem Pulsformungsfilter

In [ ]:
st = np.convolve(sym_up, g, mode='full')

plt.plot(np.arange(1024+group_delay, 2048+group_delay), st[1024+group_delay:2048+group_delay]); 
plt.title("s(t)"); plt.grid(); plt.show()

## AWGN-Kanal

In [ ]:
sigma2 = 10**(SNR/10)
nt = 1/(np.sqrt(sigma2))*np.random.randn(len(st))
snt = st + nt

plt.plot(np.arange(1024+group_delay, 2048+group_delay), snt[1024+group_delay:2048+group_delay]); 
plt.title("s(t)+n(t)"); plt.grid(); plt.show()

## Faltung mit Matched Filter am Empfänger

In [ ]:
rt = np.convolve(snt, g, mode='full')

plt.plot(np.arange(1024+2*group_delay, 2048+2*group_delay), rt[1024+2*group_delay:2048+2*group_delay]); 
plt.title("r(t) after matched filtering"); plt.grid(); plt.show()

## Abtastung, Rekonstruktion der gesendeten Symbole

In [ ]:
r = rt[2*group_delay:-2*group_delay:M]

plt.stem(r[:20]); 
plt.title("RX symbols after sampling"); plt.show()

# Symbolfehlerrate

In [ ]:
sym_est = [1 if i > 0 else -1 for i in r[:N]]
SER = np.mean([tx!=rx for tx, rx in zip(sym, sym_est)])
print("Symbol error rate: "+str(SER))